In [ ]:
import autograd.numpy as np
import capytaine as cpy
from capytaine.io.meshio import load_from_meshio
import matplotlib.pyplot as plt
import xarray as xr

import wecopttool as wot

## set colorblind-friendly colormap for plots
plt.style.use('tableau-colorblind10')

In [ ]:
wavefreq = 0.3 # Hz
f1 = wavefreq
nfreq = 10

freq = wot.frequency(f1, nfreq, False) # False -> no zero frequency

In [ ]:
amplitude = 0.0625 # m
phase = 30 # degrees
wavedir = 0 # degrees

waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

In [ ]:
wb = wot.geom.WaveBot()  # use standard dimensions
mesh_size_factor = 0.2 # 1.0 for default, smaller to refine mesh
mesh = wb.mesh(mesh_size_factor)

# create mesh object for WaveBot and add internal lid
mesh_obj = load_from_meshio(mesh, 'WaveBot')
lid_mesh = mesh_obj.generate_lid(-2e-2)

fb = cpy.FloatingBody(mesh=mesh_obj, lid_mesh=lid_mesh, name="WaveBot")

In [ ]:
fb.add_translation_dof(name="Heave")
ndof = fb.nb_dofs

In [ ]:
bem_data = wot.run_bem(fb, freq)

In [ ]:
hd = wot.add_linear_friction(bem_data, friction=500) 
# we're not actually adding friction, but need the datavariables in hd 
hd = wot.check_radiation_damping(hd)

intrinsic_impedance = wot.hydrodynamic_impedance(hd)
fig, axes = wot.utilities.plot_bode_impedance(intrinsic_impedance,
                                              'WaveBot Intrinsic Impedance')

In [ ]:
ptos = {
    'unstructured': wot.pto.PTO(ndof,
                                kinematics=np.eye(ndof),
                                controller=None,
                                impedance=None,
                                loss=None,
                                names=["PTO_Heave",]),
    'pi': wot.pto.PTO(ndof,
                      kinematics=np.eye(ndof),
                      #   controller=wot.pto.controller_pi, #TODO
                      controller=wot.controllers.pid_controller(1,True,True,False),
                      impedance=None,
                      loss=None,
                      names=["PTO_Heave",]),
    'p': wot.pto.PTO(ndof,
                     kinematics=np.eye(ndof),
                     #   controller=wot.pto.controller_pi, #TODO
                     controller=wot.controllers.pid_controller(1,True,False,False),
                     impedance=None,
                     loss=None,
                     names=["PTO_Heave",]),
}

In [ ]:
# Constraint
f_max = 750.0
nsubsteps = 4

wecs = {}
obj_funs = {}
nstate_opts = {}
for key in ptos:
    
    def const_f_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
        f = ptos[key].force(wec, x_wec, x_opt, waves, nsubsteps)
        return f_max - np.abs(f.flatten())

    constraints = [{'type': 'ineq', 'fun': const_f_pto,}]
    
    wecs[key] = wot.WEC.from_bem(
        hd,
        constraints=constraints,  # TODO
        friction=None,
        f_add={'PTO': ptos[key].force_on_wec})
    obj_funs[key] = ptos[key].mechanical_average_power
    if key == 'pi':
        nstate_opts[key] = 2
    elif key == 'p':
        nstate_opts[key] = 1
    elif key == 'unstructured':
        nstate_opts[key] = 2*nfreq

In [ ]:
pto_ds_list = []
wec_ds_list = []

results = {}
for key in wecs:
    print('\n-------------------------------')
    print(f'Running case: {key} controller')
    results[key] = wecs[key].solve(
        waves, 
        obj_funs[key], 
        nstate_opts[key],
        optim_options={'maxiter': 200}, 
        # x_wec_0=np.ones(wec.nfreq*2)*1e-1,
        # x_opt_0=np.ones(wec.nfreq*2)*1e-1,
        # scale_x_wec=1e1,
        # scale_x_opt=1e-3,
        # scale_obj=1e0,
        )
    
    x_wec, x_opt = wecs[key].decompose_state(results[key][0].x)
    nsubsteps = 5
    pto_f, pto_t = ptos[key].post_process(wecs[key], results[key], waves, nsubsteps=nsubsteps)
    wec_f, wec_t = wecs[key].post_process(wecs[key], results[key], waves, nsubsteps=nsubsteps)
    
    pto_ds_list.append(pto_t[0].expand_dims({'controller':[key]}))
    wec_ds_list.append(wec_t[0].expand_dims({'controller':[key]}))
    
    # opt_mechanical_average_power = results[0].fun
    # print(f'Optimal average mechanical power: {opt_mechanical_average_power} W')
    
pto_ds = xr.merge(pto_ds_list)
wec_ds = xr.merge(wec_ds_list)

In [ ]:
fig, ax = plt.subplots(nrows=3, sharex=True)
wec_ds['pos'].plot(ax=ax[0],
                   hue='controller',
                   add_legend=True)
ax[0].set_ylabel('Position [m]')

pto_ds['force'].plot(ax=ax[1],
                     hue='controller',
                     add_legend=False)
ax[1].set_ylabel('Force [N]')

pto_ds['power'].sel(type='mech').squeeze().plot(ax=ax[2],
                                                hue='controller',
                                                add_legend=False)

for axi in ax:
    axi.set_title('')
    axi.label_outer()
    axi.spines[['right', 'top']].set_visible(False)
    axi.autoscale(enable=True, axis='x', tight=True)

In [ ]:
pto_ds.squeeze().power.sel(type='mech').mean(dim='time')